# Tow-stage Least Squares (2SLS)

In this notebook we will see how to estimate a two stage least squares regression that addresses the posibility of (price) endogeneity.

## Pakages Used

In [1]:
# Packages used

#to upload the data:
import requests, json 

#for plotting:
import matplotlib.pyplot as plt 

# Dataframes and math:
import numpy as np
import pandas as pd
import math

# For linear regression
import statsmodels.formula.api as smf

In [2]:
# BTW, if statsmodels is not installed on your computer, run the line before once (uncomment it, run it and then comment it again)
#!pip install statsmodels

 ## Uploading the Data

In [6]:
url = "https://raw.githubusercontent.com/ArieBeresteanu/Econ1193_Spring2025/main/data/cardata2005.json"
res = requests.get(url).json()
cars = pd.DataFrame(res)

n_cars, n_cols = cars.shape

print(f"The data contains information on {n_cars} cars and it has [n_cols] columns.")

The data contains information on 217 cars and it has [n_cols] columns.


In [8]:
cars.head()

,car,year,firm_id,firm_name,division,model,hybrid,segm1,Quantity,Price,wheel_base,length,width,weight,disp,hp,mpg_city,mpg_highway,,__1
0,0,2005,3,HONDA,Acura,MDX,0,39,57948,36970,106.3,188.7,77.0,4451,3.5,265,17,23,,
1,0,2005,8,BMW,BMW,X3,0,39,30769,30995,110.1,179.7,73.0,4001,2.5,184,17,24,,
2,0,2005,8,BMW,BMW,X5,0,39,37598,42395,111.0,183.7,73.7,4652,3.0,225,15,21,,
3,0,2005,19,GM,Buick,Rainier,0,34,15271,35765,113.0,193.4,75.4,4442,4.2,275,16,21,,
4,0,2005,19,GM,Buick,Rendezvous,0,38,60589,27270,112.2,186.5,73.6,4024,3.4,185,19,26,,


## Creating New Variables

In [11]:
# using a lambda function
cars['category'] = cars['segm1'].map(lambda x: math.floor((x)/10))
# using a dictionary

categoryDict = {
    '0': 'passenger cars',
    '2': 'minivans',
    '3': 'SUV',
    '4': 'light trucks'   
}
cars['categoryName'] = cars['category'].map(lambda x: categoryDict[str(x)])

carCat = pd.crosstab(index=cars['categoryName'], columns='count')
carCat

col_0,count
categoryName,
SUV,71
light trucks,14
minivans,16
passenger cars,116


**Note:** To retrieve a value from the above dataframe, you could use the ```.loc``` method. Notice that the above dataframe (the way Python creates it) has no numerical index. The index, in the above dataframe, is the ```categoryName```. So suppose we would like to access the count for minivans. It is the third row in the second column. The name of the third row is ```minivans``` and the name of the second column is ```count```. The pair ```['SUV','count']``` defines the location in the dataframe. 

In [14]:
carCat.loc['minivans','count']

16

In [16]:
# Combined mpg

cars['mpg_combined'] = cars['mpg_city']*0.55+cars['mpg_highway']*0.45

In [18]:
# foot print in 100s of square inches 

cars['footprint'] = cars['width'] * cars['length'] /1000  #rescaling

## Type of Instrumental Variables (IVs)

### Cost shifters

When estimating a demand function the "textbook" instrumental variables are called "_cost shifters_". 

For example, Input prices (materials, labor) do not change the demand (uncorrelated with $\xi_j$) but will be correlated with the price $P_j$.

Problem: These cost shifters do not change from manufacturer to manufacturer. This kind of instrumental variable will be close to constant (weak). Moreover, cost data is rare.

### Product positioning 

Assume that the location of products in the characteristics space is exogenous, or at least determined prior
to the revelation of the consumers' valuation of the unobserved product characteristics. 

BLP use the observed product characteristics (excluding price), the sums of the values of the same characteristics of other products offered by that firm (if the firm produces more than one
product), and the sums of the values of the same characteristics of products offered by other firms.

### How many IVs?

We need to have as many IVs as endogeneous variables. In our case, we are going to assume that there is only one endogeneous variable: $P_j$.

The case were we have as many IVs as endogeneous variables is called **exactly identified**. The case where we have more IVs tha insrumental variables is called **Over identified**.

## Constructing IVs

We are going to adopt the product positioning IVs approach. The IVs we will use are defined as follows:

1. **Competition group:** For each product $j$ (we have 185 of them in our data), Let $I_j$ be the indices of all the products which are in the same category (we have four categories) as product $j$ except product $j$ itself. 
2. **Distance to Competitors**: Take an average the features of the products in the competition group $I_j$ and measure their distance to the features of product $j$. Take this distance as the instruments.

**Example:** for simplicity, let's assume that products 1,2,3 and 4 are in the same category (and only these products). Suppose we want to construct the IV for product 1. In this case, $I_1 =\{2,3,4\}$. This is product 1's competition. Suppose also that we are condidering just one feature, $hp$. Then, the instrument for $p_1$ will be $( \frac{hp_2+hp_3+hp_4}{3} -hp_1)^2$ .

From a practical point of view we can take the distance of $hp_1$ to the average of the whole category including itself. It will be easier to compute, as you will see.

### Choosing the features

In [24]:
characteristics = ['mpg_combined','footprint', 'hp', 'disp', 'weight']

### Category Average

In [27]:
featuresAvg = cars.groupby(['categoryName'])[characteristics].mean()
featuresAvg

,mpg_combined,footprint,hp,disp,weight
categoryName,,,,,
SUV,19.326056,14.134972,229.126761,3.714085,4313.521127
light trucks,20.600000,15.196911,218.142857,3.835714,4128.500000
minivans,19.871875,15.446336,201.500000,3.706250,4337.000000
passenger cars,25.532328,13.089287,192.387931,2.830172,3261.431034


The above dataframe maps a category name to the average value of the features of the cars in that category. The next step is to use these averages to create the distance of each car to the average of its competition. 

This will be done in 3 steps:
1. expand the averages from the table above to full length data frame column such that in each row the approprite average will be placed. 
2. Take a difference between each observation and the appropriate average.
3. Square the difference.


**Note:** In what follows I propose two versions to what is the "appropriate average". We can do it is the two following ways:
1. For each car take the average of the features of the cars in its category, _including_ the car in question.
2. For each car take the average of the features of the cars in its category, _excluding_ the car in question.

The first is slightly easier to implement. As you'll see below, the difference between the two methods (judged by the $R^2$ of the first stage regression), is quite minimal.

Let's demonstrate it with a speciffic example: ```hp```.

In [32]:
# 1. expanding the average hp to a column with a name hpAvg

cars['hpAvg'] = cars['categoryName'].map(lambda x: featuresAvg['hp'][x])

In [34]:
# Here is how it looks like:

cars[['categoryName','hpAvg']]

,categoryName,hpAvg
0,SUV,229.126761
1,SUV,229.126761
2,SUV,229.126761
3,SUV,229.126761
4,SUV,229.126761
...,...,...
212,passenger cars,192.387931
213,passenger cars,192.387931
214,passenger cars,192.387931
215,passenger cars,192.387931


In [36]:
# 2. Take a difference

cars['hpDist'] = cars['hp'] - cars['hpAvg']

#3. Squaring

cars['hpDist'] = cars['hpDist'].map(lambda x: x*x)

In [38]:
# Here is how it looks like:

cars[['categoryName','hp', 'hpAvg', 'hpDist']]

,categoryName,hp,hpAvg,hpDist
0,SUV,265,229.126761,1286.889308
1,SUV,184,229.126761,2036.424519
2,SUV,225,229.126761,17.030153
3,SUV,275,229.126761,2104.354096
4,SUV,185,229.126761,1947.170998
...,...,...,...,...
212,passenger cars,170,192.387931,501.219456
213,passenger cars,168,192.387931,594.771180
214,passenger cars,168,192.387931,594.771180
215,passenger cars,168,192.387931,594.771180


This is rather complicated so I wrote a function that do that for all the features you choose.

#### First version

Take an average of the features of a car's category including the car itself

In [42]:
def dist2Cat(characteristics):
    #characteristics is a list of strings. Each string in the list is a name of a characteristic
    for ch in characteristics:
        # 1. expand
        cars[ch+'Avg'] = cars['categoryName'].map(lambda x: featuresAvg[ch][x])
        # 2. difference
        cars[ch+'Dist'] = cars[ch]-cars[ch+'Avg']
        # 3. square
        cars[ch+'Dist'] = cars[ch+'Dist'].map(lambda x: x*x)
        # 4. Refferential outlook 
        cars[ch+'Avg'] = 
        

In [44]:
dist2Cat(characteristics)

In [46]:
cars

,car,year,firm_id,firm_name,division,model,hybrid,segm1,Quantity,Price,...,hpAvg,hpDist,mpg_combinedAvg,mpg_combinedDist,footprintAvg,footprintDist,dispAvg,dispDist,weightAvg,weightDist
0,0,2005,3,HONDA,Acura,MDX,0,39,57948,36970,...,229.126761,1286.889308,19.326056,0.139834,14.134972,0.155968,3.714085,0.045832,4313.521127,18900.440587
1,0,2005,8,BMW,BMW,X3,0,39,30769,30995,...,229.126761,2036.424519,19.326056,0.678883,14.134972,1.034029,3.714085,1.474001,4313.521127,97669.454672
2,0,2005,8,BMW,BMW,X5,0,39,37598,42395,...,229.126761,17.030153,19.326056,2.644059,14.134972,0.355552,3.714085,0.509917,4313.521127,114567.947629
3,0,2005,19,GM,Buick,Rainier,0,34,15271,35765,...,229.126761,2104.354096,19.326056,1.157897,14.134972,0.200156,3.714085,0.236114,4313.521127,16506.820869
4,0,2005,19,GM,Buick,Rendezvous,0,38,60589,27270,...,229.126761,1947.170998,19.326056,7.974658,14.134972,0.166931,3.714085,0.098649,4313.521127,83822.482841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,1,2005,7,VOLKS,volkswagen,passat,0,3,49233,24955,...,192.387931,501.219456,25.532328,0.232640,13.089287,0.133990,2.830172,1.061255,3261.431034,25782.392687
213,1,2005,6,VOLVO,volvo,S40,0,3,24241,23945,...,192.387931,594.771180,25.532328,5.675485,13.089287,0.687335,2.830172,0.185048,3261.431034,31481.771998
214,1,2005,6,VOLVO,volvo,s60,0,3,24695,27920,...,192.387931,594.771180,25.532328,11.440140,13.089287,0.087076,2.830172,0.185048,3261.431034,160455.496136
215,1,2005,6,VOLVO,volvo,v70 c70 s70,0,4,22823,29445,...,192.387931,594.771180,25.532328,11.440140,13.089287,0.005493,2.830172,0.185048,3261.431034,34807.978894


#### Second version

Take an average of the features of a car's category excluding the car itself

In [49]:
cars['categoryCount'] = cars['categoryName'].map(lambda x: carCat.loc[x,'count'])

In [51]:
def dist2CatV2(characteristics):
    #characteristics is a list of strings. Each string in the list is a name of a characteristic
    for ch in characteristics:
        # 1. expand
        #cars[ch+'Avg'] = cars['categoryName'].map(lambda x: featuresAvg[ch][x])
        cars[ch+'Avg2'] = (cars[ch+'Avg']*cars['categoryCount'] - cars[ch])/(cars['categoryCount']-1)
        # 2. difference
        cars[ch+'Dist2'] = cars[ch]-cars[ch+'Avg2']
        # 3. square
        cars[ch+'Dist2'] = cars[ch+'Dist2'].map(lambda x: x*x)
        

In [53]:
dist2CatV2(characteristics)

In [55]:
cars

,car,year,firm_id,firm_name,division,model,hybrid,segm1,Quantity,Price,...,mpg_combinedAvg2,mpg_combinedDist2,footprintAvg2,footprintDist2,hpAvg2,hpDist2,dispAvg2,dispDist2,weightAvg2,weightDist2
0,0,2005,3,HONDA,Acura,MDX,0,39,57948,36970,...,19.320714,0.143858,14.129330,0.160456,228.614286,1323.920204,3.717143,0.047151,4311.557143,19444.310408
1,0,2005,8,BMW,BMW,X3,0,39,30769,30995,...,19.314286,0.698418,14.149499,1.063783,229.771429,2095.023673,3.731429,1.516416,4317.985714,100479.943061
2,0,2005,8,BMW,BMW,X5,0,39,37598,42395,...,19.349286,2.720143,14.143490,0.365783,229.185714,17.520204,3.724286,0.524590,4308.685714,117864.698776
3,0,2005,19,GM,Buick,Rainier,0,34,15271,35765,...,19.341429,1.191216,14.128581,0.205916,228.471429,2164.907959,3.707143,0.242908,4311.685714,16981.813061
4,0,2005,19,GM,Buick,Rendezvous,0,38,60589,27270,...,19.285714,8.204133,14.140809,0.171735,229.757143,2003.201837,3.718571,0.101488,4317.657143,86234.517551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,1,2005,7,VOLKS,volkswagen,passat,0,3,49233,24955,...,25.536522,0.236703,13.092470,0.136331,192.582609,509.974216,2.839130,1.079792,3260.034783,26232.731645
213,1,2005,6,VOLVO,volvo,S40,0,3,24241,23945,...,25.553043,5.774618,13.096496,0.699341,192.600000,605.160000,2.833913,0.188281,3262.973913,32031.661550
214,1,2005,6,VOLVO,volvo,s60,0,3,24695,27920,...,25.561739,11.639964,13.091853,0.088597,192.600000,605.160000,2.833913,0.188281,3257.947826,163258.159244
215,1,2005,6,VOLVO,volvo,v70 c70 s70,0,4,22823,29445,...,25.561739,11.639964,13.088642,0.005589,192.600000,605.160000,2.833913,0.188281,3259.808696,35415.967032


## First stage regression

We want to regress the price on all the features **and** the instrumental variables we created. The tree features we will use are ```hp```, ```mpg_combined``` and ```footprint```. We add the three IVs associated with these features. 

#### First version:

In [59]:
firstStageV1 = smf.ols(formula='Price ~   disp + mpg_combined + footprint + dispDist + mpg_combinedDist + footprintDist',data=cars).fit()
print(firstStageV1.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.384
Method:                 Least Squares   F-statistic:                     23.40
Date:                Mon, 21 Apr 2025   Prob (F-statistic):           4.18e-21
Time:                        19:33:39   Log-Likelihood:                -2321.5
No. Observations:                 217   AIC:                             4657.
Df Residuals:                     210   BIC:                             4681.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         4.812e+04    1.3e+04  

We had the predicted values of the price to the dataframe as a new column.

In [62]:
cars['Price_hatV1'] = firstStageV1.predict()

#### Second version:

In [65]:
firstStageV2 = smf.ols(formula='Price ~   disp + mpg_combined + footprint + dispDist2 + mpg_combinedDist2 + footprintDist2',data=cars).fit()

print(firstStageV2.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                     23.39
Date:                Mon, 21 Apr 2025   Prob (F-statistic):           4.30e-21
Time:                        19:33:39   Log-Likelihood:                -2321.5
No. Observations:                 217   AIC:                             4657.
Df Residuals:                     210   BIC:                             4681.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept          4.753e+04    1.3e+0

In [99]:
cars['Price_hatV2'] = firstStageV2.predict()

As one can see, the $R^2$ in both versions is quite the same.

#### A note on weak IVs

As we said, the IV should be uncorrelated with the unobserved characteristic $\xi$ (exogeneity) AND correlated with the price $P_j$ (relevance). The uncorrelation with $\xi$ cannot be tested because $\xi$ is unobserved! The truthfulness of this assumption depends on economic/econometric arguments. The correlation with $P_j$ can be mesured from the first stage regression. A commonly used rule of thumb is that the $R^2$ in the first stage regression should be larger than $0.1$. We can also see that at least two of the IVs come out significant in the first stage regression. In other words, these are not weak instrunments.

the weak isntruments can justify weak and strong econometric 

The next step is to use ```Price_hat``` in the second stage regression instead of using ```Price```.